As prototypes evolve into robust software, one common challenge is to smoothly phase out less scalable development patterns.  In the world of the tidyverse, scripts often leverage the `cli` package for logging purposes.  As things evolve and these scripts are refactored, it may be useful to fold `cli` output into a more structured logging environment.  `metayer` makes this transition relatively painless, and this vignette walks through the process.

### load the metayer package 

In [ ]:
# load metayer package
suppressMessages(
  devtools::load_all()
)

## logger basics

The logger package documentation is at [https://daroczig.github.io/logger](https://daroczig.github.io/logger).

Logs are organized by namespace, the idea being that some base configuration is adapted across new namespaces.  This makes it relatively easy to manage multiple streams of logs.  To start, the pattern is typically:

In [ ]:
log_threshold(INFO)
log_appender(appender_stdout) # in jupyter, for example
log_info("hello world")

If we want to change the layout, say, to show the namespace:

In [ ]:
log_layout(
  layout_glue_generator(
    format = "{ns} {level} [{format(time, \"%Y-%m-%d %H:%M:%S\")}] {msg}"
  )
)
log_info("a new format, with namespaces")

Or, perhaps we want send logs to a file.  This can be accomplished with a new, "logfile", namespace.

In [ ]:
tmp <- tempfile()

log_appender(
    appender_file(tmp),
    namespace = "logfile"
)
log_info("to the filesystem", namespace = "logfile")

# read from the temporary file
msg <- xfun::read_utf8(tmp)
cat(msg, "\n")

## cli

For scripts that already leverage the `cli` package, `metayer` provides some easy shims to redirect cli messages into logger.  These shims use the `cli.default_handler` hook:

In [ ]:
options(cli.default_handler = logged_cli_handler)

As before,

In [ ]:
logger_reset()
log_info("hello world")

However, calling `cli_text`, a wrapped version of `cli::cli_text` produces output on two new namespaces:

In [ ]:
cli_text("a cli output")

The first is from a function in the `metayer` package namespace; in fact, the `logged_cli_handler` function.  The second is to a namespace called `global.cli` (*sic*) which indicates that a cli function was called in the global environment.  Or at least that's what you'd see if you were in an interpreter.  However, what you'll see in this document is `metayer.cli`, as the vignette is built in the package namespace, not the global namespace.

### lower level logging

Suppose we wanted to examine lower level logging but only in the `metayer` package.  The following will suffice:

In [ ]:
log_threshold(TRACE, namespace = "metayer")
cli_text("a cli output")

# restore the previous threshold
log_threshold(INFO, namespace = "metayer")

## appendix:  wrapping via metaprogramming

Each `cli` function is wrapped with logic that captures conditions and sends the corresponding condition messages into the `logger` machinery.  This all happens in the `cli-wrapped.R` source file, and there are scripts that automate this process.  In particular, these scripts generate entries like the following:

```R
cli_alert <- cli_wrap_safe("cli_alert", logger::INFO)
```

where `cli_wrap_safe` manages the details.  The wrapping logic lives in the `wrap_cli_body` function.  This is interesting because it's called both directly and via a metaprogramming substitution.

In [ ]:
code <- capture_output(print(wrap_cli_body)) %>%
    gsub("<environment: .*>", "", .)
display_source(
    code,
    raw = TRUE
)

Compare, for example, to the implementation of our wrapped `cli_text` function.  Note the substitution within the conditional block that generates `outer_cnd`.  `cmd` is replaced by `cli::cli_text(..., .envir = .envir)`.

In [ ]:
code <- deparse(cli_text) %>% paste0(collapse = "\n")
display_source(
    code,
    raw = TRUE
)